## 웹캠 사용 및 랜드마크 수집을 위한 사전설정

In [1]:
import time, pickle
import cv2
import mediapipe as mp
import numpy as np
from pynput.mouse import Listener
from screeninfo import get_monitors
# based on https://github.com/google/mediapipe/blob/master/docs/solutions/face_mesh.md

def on_click(x, y, button, pressed):
    # 마우스 클릭될 때마다 좌표와 시간, 얼굴 랜드마크를 기록하는 함수
    monitor = get_monitors()[0]
    if pressed and results.multi_face_landmarks:
        records.append(
            {
                'coord' : (x/monitor.width, y/monitor.height),
                'time' : time.time()-start_time,
                'frame' : frame_count,
                'landmark' : np.array([ [d.x, d.y, d.z] for d in results.multi_face_landmarks[0].landmark ])
            }
        )
        print(f"Mouse clicked at: {x/monitor.width}, {y/monitor.height} \t time: {time.time()-start_time:0.2f}")
        # save with pickle
        with open('records.pkl', 'wb') as f:
            pickle.dump(records, f)
        
def count_captured_landmarks():
    # 전체 landmark중 얼마나 화면에 찍히고 있는지 알려주는 함수
    count = 0 
    for data_point in results.multi_face_landmarks[0].landmark:
        if (data_point.x <= 1 and data_point.x >= 0) and (data_point.y <= 1 and data_point.y >= 0):
            count += 1
    return count

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

records = [] # 클릭 좌표를 저장할 리스트
frame_count = 0 # 프레임 수를 저장할 변수
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

start_time = time.time()
listener = Listener(on_click=on_click)
listener.start()
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# 웹캠 화면이 최상단에 뜨게 하기 위한 설정
# 최상단을 유지하려는 경우에만 주석을 해제해주세요
# cv2.namedWindow('window', cv2.WINDOW_AUTOSIZE)
# cv2.setWindowProperty('window', cv2.WND_PROP_TOPMOST, 1)

## 웹캠 실행

In [2]:
with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("웹캠을 찾을 수 없습니다.")
            break

        image.flags.writeable = False # 성능 향상을 위해 이미지를 읽기 전용으로 만듭니다.
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_tesselation_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_contours_style())
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_IRISES,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp_drawing_styles
                    .get_default_face_mesh_iris_connections_style())
                
            image = cv2.flip(image, 1)
            cv2.putText(image, f'{count_captured_landmarks()}/478', (5, 25), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
        cv2.imshow('window', image)
        # 이미지 재생을 위한 대기시간
        cv2.waitKey(5)
        frame_count += 1
        
cap.release()

Mouse clicked at: 0.26145833333333335, 0.3074074074074074 	 time: 2.69
Mouse clicked at: 0.28802083333333334, 0.5574074074074075 	 time: 5.47
Mouse clicked at: 0.5567708333333333, 0.4583333333333333 	 time: 7.26
Mouse clicked at: 0.19322916666666667, 0.6351851851851852 	 time: 9.40
Mouse clicked at: 0.6322916666666667, 0.5777777777777777 	 time: 10.70
Mouse clicked at: 0.10416666666666667, 0.07592592592592592 	 time: 14.02


KeyboardInterrupt: 

## Record 형식 확인

In [ ]:
records[-1]

{'coord': (335, 191),
 'time': 23.87341046333313,
 'frame': 232,
 'landmark': array([[ 0.73864388,  1.05022252, -0.02767511],
        [ 0.7614361 ,  0.99460292, -0.06176444],
        [ 0.75393409,  1.00860226, -0.03121721],
        ...,
        [ 0.80475986,  0.85804492,  0.0242392 ],
        [ 0.79443681,  0.87345934,  0.0242392 ],
        [ 0.80764079,  0.88528502,  0.0242392 ]])}

Mouse clicked at: 1919, 807 	 time: 50.86
Mouse clicked at: 1122, 1059 	 time: 53.83
Mouse clicked at: 420, 450 	 time: 55.26
Mouse clicked at: 934, 206 	 time: 56.04
Mouse clicked at: 235, 133 	 time: 56.82
Mouse clicked at: 293, 519 	 time: 57.86
Mouse clicked at: 1150, 573 	 time: 59.28
Mouse clicked at: 1616, 231 	 time: 60.18
Mouse clicked at: 884, 1079 	 time: 61.39
Mouse clicked at: 1466, 14 	 time: 62.39
Mouse clicked at: 394, 365 	 time: 69.21
Mouse clicked at: 847, 429 	 time: 71.16
Mouse clicked at: 967, 1068 	 time: 72.41
Mouse clicked at: 387, 554 	 time: 73.52
Mouse clicked at: 383, 640 	 time: 74.14
Mouse clicked at: 527, 502 	 time: 74.66
Mouse clicked at: 847, 750 	 time: 75.61
Mouse clicked at: 643, 170 	 time: 76.34
Mouse clicked at: 443, 0 	 time: 76.79
Mouse clicked at: 1186, 402 	 time: 77.91
Mouse clicked at: 1136, 1061 	 time: 79.45
Mouse clicked at: 1131, 1062 	 time: 81.51
Mouse clicked at: 824, 600 	 time: 97.89
Mouse clicked at: 1138, 1054 	 time: 126.15
Mou


## Landmark 형태 확인

In [ ]:
landmarks = [ r['landmark'] for r in records ]
landmarks = np.array(landmarks)
print(landmarks.shape)

(45, 478, 3)


Mouse clicked at: 349, 714 	 time: 184.95
Mouse clicked at: 183, 780 	 time: 186.76
Mouse clicked at: 941, 1079 	 time: 189.79
Mouse clicked at: 982, 527 	 time: 190.83
Mouse clicked at: 982, 527 	 time: 190.97
Mouse clicked at: 525, 176 	 time: 191.99
Mouse clicked at: 521, 161 	 time: 192.46
Mouse clicked at: 913, 1066 	 time: 195.86
Mouse clicked at: 1312, 4 	 time: 196.61
Mouse clicked at: 838, 0 	 time: 197.00
Mouse clicked at: 828, 0 	 time: 197.36
Mouse clicked at: 823, 5 	 time: 197.95
Mouse clicked at: 573, 0 	 time: 198.99
Mouse clicked at: 558, 46 	 time: 199.30
Mouse clicked at: 345, 605 	 time: 205.92
Mouse clicked at: 1079, 1068 	 time: 210.10
Mouse clicked at: 800, 532 	 time: 210.64
Mouse clicked at: 495, 184 	 time: 211.55
Mouse clicked at: 769, 372 	 time: 215.49
Mouse clicked at: 1087, 1055 	 time: 217.67
Mouse clicked at: 1091, 1051 	 time: 218.88
Mouse clicked at: 1060, 1060 	 time: 219.82
Mouse clicked at: 1010, 1057 	 time: 220.31
Mouse clicked at: 199, 908 	 tim